We’ll use the 'Russian housing market' dataset from Kaggle.
First, let’s take a look at the data in Python.

In [ ]:
import pandas as pd
df = pd.read_csv('bank-data.csv') # renamed from the csv file within train.csv.zip on Kaggle

df.info(verbose=True)

df.head()

We can also identify the numeric and non-numeric columns. These are necessary since we often treat them using different methods.

In [ ]:
numeric_cols = df.select_dtypes(include=['number']).columns
print(numeric_cols)

non_numeric_cols = df.select_dtypes(exclude=['number']).columns
print(non_numeric_cols)

## Missing data
Missing data or missing values are when there’s no data value stored for a column in a row. It is very common among real-world datasets. If not handled properly, they would significantly impact the results of data analysis. Many machine learning models can’t tolerate any missing values. So, we must learn how to handle missing data.

#### Method #1: missing data (by columns) count & percentage
This is the most basic method to detect missing data among columns.
We print out the summary of all the non-numeric columns below. By looking at the Non-Null Count, we can spot the number of missing data by columns. For our example, all non-null counts are 30,471, the same as the total number of rows, so there are no missing data among the non-numeric columns!

In [ ]:
df[non_numeric_cols].info()

We can print out the first 10 columns to look at.

In [ ]:
num_missing = df.isna().sum()
num_missing[:10]

In [ ]:
pct_missing = df.isna().mean()
pct_missing[:10]

#### Method #2: missing data (by columns) heatmap
The seaborn library is a popular statistical data visualization library. Let’s first use it to plot the missing data heatmap.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,8))

cols = df.columns[:30]
colours = ['#000099', '#ffff00'] # specify colours: yellow - missing. blue - not missing
sns.heatmap(df[cols].isna(), cmap=sns.color_palette(colours))

#### Method #3: missing data (by rows) histogram
We’ve been looking at missing data by columns. But we can also summarize the missing data by rows. Missing data histogram is a technique for summarizing such information.

In [ ]:
missing_by_row = df.isna().sum(axis='columns')
missing_by_row.hist(bins=50)

### What to do?
There are NO agreed-upon solutions to dealing with missing data. We have to study the dataset and its specific columns to determine how to clean their missing values.

#### Technique #1: drop columns / features
We drop the entire column or feature with missing data, which will certainly cause a loss of information. So we should only perform this when we are sure that the missing data is not informative. 

In [ ]:
pct_missing[pct_missing > .3] # What are the columns with over 30% missing data then apply the copy method to make a copy to the new DataFrame df_less_missing_cols. 
df_less_missing_cols = df.loc[:, pct_missing <= .3].copy() # equivalent to df.drop(columns=pct_missing[pct_missing > .3].index)
df_less_missing_cols.shape

#### Technique #2: drop rows / observations
We can drop the entire row with missing data like the first technique. Again, please be aware of the loss of information when removing rows.

In [ ]:
df_less_missing_rows = df[missing_by_row < 35].copy()
df_less_missing_rows.shape # equivalent to df.dropna(axis='index', thresh=292-35+1).shape

#### Technique #3: impute the missing with constant values
Instead of dropping data, we can also replace the missing. An easy method is to impute the missing with constant values. For example, we can impute the numeric columns with a value of -999 and impute the non-numeric columns with ‘_MISSING_’.

In [ ]:
df_copy = df.copy()
df_copy[numeric_cols] = df_copy[numeric_cols].fillna(-999)
df_copy[non_numeric_cols] = df_copy[non_numeric_cols].fillna('_MISSING_')

#### Technique #4: impute the missing with statistics
Besides constants, we can also impute the missing values with statistics.
For example, we can impute the numeric columns with their respective medians.

In [ ]:
df_copy = df.copy()
med = df_copy[numeric_cols].median()
df_copy[numeric_cols] = df_copy[numeric_cols].fillna(med)

We can also impute the non-numeric columns with their most frequent values. Then we can use it to fill in the missing.

In [ ]:
most_freq = df_copy[non_numeric_cols].describe().loc['top']
most_freq
df_copy[non_numeric_cols] = df_copy[non_numeric_cols].fillna(most_freq)

## Irregular data (outliers)
Outliers are data that is distinct from other observations. They could bias our data analysis results, providing a misleading representation of the data. Outliers could be real outliers or mistakes.

#### Method #1: descriptive statistics
First, let’s look at kurtosis. Kurtosis is a statistical measure of ‘tailedness’. The higher kurtosis is often linked to the greater extremity of deviations (or outliers) in the data. So this is a single statistic to detect potential outliers. Among the first 10 columns, we can see that life_sq has the highest kurtosis value. But note that the high value of kurtosis doesn’t guarantee outliers. So we’ll investigate this column more soon. For the column life_sq, we can see that the maximum value is 7,478, while the 75th percentile is only 43. The maximum value is an outlier.

In [ ]:
df.kurt(numeric_only=True)[:10]
df['life_sq'].describe()

#### Method #2: histogram & box plot
From the histogram, we can see that the data is highly skewed with the possible existence of outliers. But due to the low frequency, we can’t see the exact location of the outliers and the counts. From the histogram, we can see that the data is highly skewed with the possible existence of outliers. But due to the low frequency, we can’t see the exact location of the outliers and the counts. From the box plot, we get a more clear view of the outliers. There is an outlier with a value of over 7,000. In fact, all the dots on the plot are considered outliers by the box plot definition.

In [ ]:
df['life_sq'].hist(bins=100)

df.boxplot(column=['life_sq'])

### What to do?
While outliers are not hard to see, it is tricky to clean them. It depends on the dataset and the goal of the project.

The methods of handling outliers are somewhat similar to missing data. We could drop, replace, or even just keep them.

## Unnecessary data
Those are a lot of hard work for missing data and outliers! Let’s clean something more straightforward in this section: the unnecessary data.

#### Unnecessary type #1: repetitive & uninformative
One column can have many observations being the same value. When an extremely high percentage of the column has a repetitive value, we should investigate whether such a column provides valuable information.

We can generate a list of columns with a high percentage of the same value. For example, we specify below to show columns with over 99.9% rows being the same value.

In [ ]:
num_rows = len(df)

for col in df.columns:
    cnts = df[col].value_counts(dropna=False)
    top_pct = (cnts/num_rows).iloc[0]
    
    if top_pct > 0.999:
        print('{0}: {1:.2f}%'.format(col, top_pct*100))
        print(cnts)
        print()

In fact, there’s no such column in our example dataset.

#### What to do?
If there is one column with a high percentage of the same value, we should look into it to see if it’s informative. We can drop them when they are not, e.g., when the column has 100% being the same value.

#### Unnecessary type #2: irrelevant
Again, the data needs to provide valuable information for the project. If the features are not related to the question we are trying to solve, they are irrelevant.

#### How to find out?
We need to skim through the features to identify irrelevant ones. For example, a feature recording the temperature in the US wouldn’t provide direct insights into housing prices in Russia.

#### What to do?
When the features are not serving the project’s goal, we can remove them. You could use the 'drop' method in pandas.

#### Unnecessary type #3: duplicates
The duplicate data is when copies of the same observation exist. Let’s look at 2 main types of duplicate data and clean them in Python.

#### Duplicates type #1: all columns based
#### How to find out?
This is easy to understand. Such duplicate occurs when all the columns’ values within the observations are the same.

We can use the duplicated method to grab the boolean values of whether a row is duplicated, and then use it to filter for duplicated rows from df.

We can use the drop_duplicates method. But this will return the same DataFrame since there weren’t any duplicates.

In [ ]:
df[df.duplicated()]
df.drop_duplicates()

## Inconsistent data

#### Inconsistent type #1: capitalization
Inconsistent use of upper and lower cases in categorical values is typical. We need to clean it since Python is case-sensitive.

In [ ]:
df['sub_area'].value_counts(dropna=False)
df['sub_area_lower'] = df['sub_area'].str.lower()
df['sub_area_lower'].value_counts(dropna=False)

#### Inconsistent type #2: data types
Another standardization we often need to look at is the data types.

We could also print out one column to take a look. Let’s try timestamp.

In [ ]:
df['timestamp']

timestamp has dtype of object while it records dates.

In [ ]:
df['timestamp_dt'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d')
df['year'] = df['timestamp_dt'].dt.year
df['month'] = df['timestamp_dt'].dt.month
df['weekday'] = df['timestamp_dt'].dt.weekday

df[['timestamp_dt', 'year', 'month', 'weekday']].head()

#### Inconsistent type #3: typos of categorical values
A categorical column takes on a limited and usually fixed number of possible values. Sometimes it shows other values due to reasons like typos.

Let’s see an example. Within the code below:

* We generate a new DataFrame, df_city_ex
There is only one column that stores the city names. There are misspellings. For example, ‘torontoo’ and ‘tronto’ both refer to the city of ‘toronto’.
* The variable cities stores the 4 correct names of ‘toronto’, ‘vancouver’, ‘montreal’, and ‘calgary’.
* To identify typos, we use fuzzy logic matches. We use edit_distance from nltk, which measures the number of operations (e.g., substitution, insertion, deletion) needed to change from one string into another string.
* We calculate the distance between the actual values and the correct values.

In [ ]:
df_city_ex = pd.DataFrame(data={'city': ['torontoo', 'toronto', 'tronto', 'vancouver', 'vancover', 'vancouvr', 'montreal', 'calgary']})

cities = ['toronto', 'vancouver', 'montreal', 'calgary']
from nltk.metrics import edit_distance
for city in cities:
    df_city_ex[f'city_distance_{city}'] = df_city_ex['city'].map(lambda x: edit_distance(x, city))

df_city_ex

#### What to do?
We can set criteria to convert these typos to the correct values. For example, the below code sets all the values within 2 characters distance from ‘toronto’/’vancouver’ to be ‘toronto’/’vancouver’.

In [ ]:
msk = df_city_ex['city_distance_toronto'] <= 2
df_city_ex.loc[msk, 'city'] = 'toronto'

msk = df_city_ex['city_distance_vancouver'] <= 2
df_city_ex.loc[msk, 'city'] = 'vancouver'

df_city_ex

#### Inconsistent type #4: addresses
----- your code below -----